In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

tabelas = '/content/drive/Shareddrives/BTECH/BTECH/Tabelas'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Download do colab com limpeza dos dados
%run /content/drive/Shareddrives/BTECH/BTECH/limpeza_dados.ipynb

In [ ]:
#Retirar cursos de empresas
df_cursos = df_cursos.rename(columns={'id': 'journey_id'})
df_cursos.sort_values(by='journey_id', ascending=True)

In [ ]:
df_cursos

,journey_id,improve_eng,improve_int,improve_res,improve_dis,improve_cur,emp,improve_neg,improve_tech,improve_sin,plataforma
0,309,0,0,0,1,0,0,0,0,1,0
1,103,1,1,0,0,0,0,0,0,1,0
2,38,0,1,1,0,0,0,0,0,0,0
3,108,0,1,0,0,0,0,1,0,0,0
5,105,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
193,121,0,0,0,0,0,0,0,0,1,0
197,672,1,1,0,1,0,0,0,0,0,0
200,827,1,1,0,0,0,0,0,0,0,0
203,110,0,0,0,1,0,0,0,0,1,0


In [ ]:
#Juntar DataFrames

user_course_data = df_user_journeys.merge(user_objectives, left_on='user_id', right_on='id', how='inner')
user_course_data

,user_id,journey_id,id,Administrativo,Comercial / Vendas,Engenharia,Finanças / Contabilidade,Jurídico,Logistica / Supply Chain,Marketing / Comunicação,Produção / Operações,RH,Saúde,Tecnologia
0,279236,86,279236,1,1,1,1,0,1,0,1,0,0,1
1,279236,158,279236,1,1,1,1,0,1,0,1,0,0,1
2,279236,862,279236,1,1,1,1,0,1,0,1,0,0,1
3,532807,86,532807,1,1,0,1,0,1,0,0,0,0,0
4,532807,124,532807,1,1,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,1130119,828,1130119,0,0,0,0,0,0,0,0,0,0,1
10837,1130119,829,1130119,0,0,0,0,0,0,0,0,0,0,1
10838,1130119,879,1130119,0,0,0,0,0,0,0,0,0,0,1
10839,1130119,897,1130119,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
pivot_df = pd.pivot_table(user_course_data, index='journey_id', columns='id', values=['Administrativo', 'Comercial / Vendas', 'Engenharia', 'Finanças / Contabilidade', 'Jurídico', 'Logistica / Supply Chain', 'Marketing / Comunicação', 'Produção / Operações', 'RH', 'Saúde', 'Tecnologia'], aggfunc='sum', fill_value=0)

# Calculate the sums of how many users from each area have completed each course
sums_by_area = pivot_df.groupby(level=0, axis=1).sum()

# Reset the index to have courses as rows
sums_by_area.reset_index(inplace=True)

# Print the resulting DataFrame
sums_by_area

,journey_id,Administrativo,Comercial / Vendas,Engenharia,Finanças / Contabilidade,Jurídico,Logistica / Supply Chain,Marketing / Comunicação,Produção / Operações,RH,Saúde,Tecnologia
0,19,1,0,0,0,0,1,1,0,0,0,1
1,23,0,1,0,0,0,0,0,0,0,0,0
2,25,438,252,277,203,37,224,317,291,183,87,490
3,26,0,0,0,0,0,1,0,1,0,0,0
4,51,1,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
106,897,16,18,6,12,3,8,29,7,7,0,13
107,898,15,17,11,8,2,11,31,14,9,4,24
108,899,0,2,0,0,0,0,2,0,0,0,0
109,900,4,5,3,3,0,2,6,4,2,2,5


In [ ]:
#Normalização dos dados
journeys_id = sums_by_area['journey_id']
lista1 = sums_by_area.drop(labels='journey_id', axis=1)

journeys_id = sums_by_area['journey_id']
standard = StandardScaler()

# Para normalizar os dados de 0 a 1
scaler = MinMaxScaler()
lista1 = scaler.fit_transform(lista1)
df_relevance_courses = pd.DataFrame(lista1, columns=['Administrativo', 'Comercial / Vendas','Engenharia', 'Finanças / Contabilidade', 'Jurídico', 'Logistica / Supply Chain', 'Marketing / Comunicação', 'Produção / Operações', 'RH', 'Saúde', 'Tecnologia'
])

df_relevance_courses['journey_id'] = journeys_id.reset_index(drop=True)
df_relevance_courses = df_relevance_courses[['journey_id', 'Administrativo', 'Comercial / Vendas','Engenharia', 'Finanças / Contabilidade', 'Jurídico', 'Logistica / Supply Chain', 'Marketing / Comunicação', 'Produção / Operações', 'RH', 'Saúde', 'Tecnologia'
]]
df_relevance_courses

,journey_id,Administrativo,Comercial / Vendas,Engenharia,Finanças / Contabilidade,Jurídico,Logistica / Supply Chain,Marketing / Comunicação,Produção / Operações,RH,Saúde,Tecnologia
0,19,0.001869,0.000000,0.000000,0.000000,0.000000,0.003876,0.002079,0.000000,0.000000,0.000000,0.001488
1,23,0.000000,0.003344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,25,0.818692,0.842809,0.923333,0.898230,0.560606,0.868217,0.659044,0.909375,0.788793,0.690476,0.729167
3,26,0.000000,0.000000,0.000000,0.000000,0.000000,0.003876,0.000000,0.003125,0.000000,0.000000,0.000000
4,51,0.001869,0.000000,0.003333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001488
...,...,...,...,...,...,...,...,...,...,...,...,...
106,897,0.029907,0.060201,0.020000,0.053097,0.045455,0.031008,0.060291,0.021875,0.030172,0.000000,0.019345
107,898,0.028037,0.056856,0.036667,0.035398,0.030303,0.042636,0.064449,0.043750,0.038793,0.031746,0.035714
108,899,0.000000,0.006689,0.000000,0.000000,0.000000,0.000000,0.004158,0.000000,0.000000,0.000000,0.000000
109,900,0.007477,0.016722,0.010000,0.013274,0.000000,0.007752,0.012474,0.012500,0.008621,0.015873,0.007440


In [ ]:
area_compet = df_cursos.merge(df_relevance_courses, left_on='journey_id', right_on='journey_id', how='inner')
area_compet = area_compet.sort_values(by='journey_id', ascending=True)

In [ ]:
def remove_prefix(column_name):
    if column_name.startswith('improve_'):
        return column_name[len('improve_'):]
    else:
        return column_name

area_compet = area_compet.rename(columns=remove_prefix)
area_compet

,journey_id,eng,int,res,dis,cur,emp,neg,tech,sin,...,Comercial / Vendas,Engenharia,Finanças / Contabilidade,Jurídico,Logistica / Supply Chain,Marketing / Comunicação,Produção / Operações,RH,Saúde,Tecnologia
28,23,0,0,0,1,0,0,0,0,1,...,0.003344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33,26,0,0,1,0,0,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,0.003876,0.000000,0.003125,0.000000,0.000000,0.000000
10,51,0,0,0,0,0,0,0,0,1,...,0.000000,0.003333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001488
8,58,0,0,0,1,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001488
6,59,0,1,0,0,0,0,0,0,0,...,0.003344,0.000000,0.004425,0.000000,0.000000,0.002079,0.000000,0.000000,0.000000,0.001488
27,74,1,0,0,0,1,0,0,0,0,...,0.000000,0.003333,0.000000,0.000000,0.000000,0.002079,0.003125,0.000000,0.000000,0.000000
24,75,0,0,0,0,0,0,0,1,0,...,0.003344,0.000000,0.004425,0.000000,0.000000,0.002079,0.000000,0.000000,0.000000,0.001488
22,76,0,0,0,1,0,0,0,0,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001488
15,82,0,0,1,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.001488
1,105,0,0,0,0,1,0,0,0,1,...,0.006689,0.000000,0.008850,0.000000,0.003876,0.006237,0.003125,0.000000,0.007937,0.002976


In [ ]:
user_objectives

,user_id,Administrativo,Comercial / Vendas,Engenharia,Finanças / Contabilidade,Jurídico,Logistica / Supply Chain,Marketing / Comunicação,Produção / Operações,RH,Saúde,Tecnologia
0,13973,0,0,0,0,0,0,1,1,0,0,1
1,40903,1,0,0,0,0,1,0,1,0,0,0
2,46796,0,0,0,0,0,1,0,1,0,0,1
3,67266,0,0,0,1,0,0,0,1,0,0,0
4,68161,0,1,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
18620,1130075,0,0,1,0,0,0,1,0,0,0,1
18621,1130119,0,0,0,0,0,0,0,0,0,0,1
18622,1130135,0,0,0,0,0,0,1,0,0,0,0
18623,1130171,1,0,0,0,0,0,1,0,1,0,0


In [ ]:
# Set the index of user-objectives DataFrame to 'id'
user_objectives.set_index('user_id', inplace=True)

# Transpose the area_compet DataFrame to have courses as rows and competences as columns
area_compet_transposed = area_compet.set_index('journey_id').T

# Initialize and fit the NearestNeighbors model
nn_model = NearestNeighbors(n_neighbors=1, metric='cosine', algorithm='brute')
nn_model.fit(area_compet_transposed)

# Create a dictionary to store recommendations
recommendations = {}

# Iterate through user-objectives and find the most similar course for each user
for user_id, objectives in user_objectives.iterrows():
    objectives = objectives.values.reshape(1, -1)
    _, indices = nn_model.kneighbors(objectives)
    course_id = area_compet_transposed.iloc[indices[0][0]].name
    recommendations[user_id] = course_id

# Display the recommendations
for user_id, course_id in recommendations.items():
    print(f"User {user_id} should consider Course {course_id}")


ValueError: ignored

Várias tabelas com a área os cursos mais relevantes para aquela área